In [1]:
#Imports
import time
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
import urllib.request
from urllib.request import urlopen
import pyautogui
from requests.exceptions import ConnectionError
# from pywebcopy import save_webpage

In [2]:

internal_urls = set()
external_urls = set()
def is_url_valid(url):
   #Checks if the url is valid or not
    parsedURL = urlparse(url)
    return bool(parsedURL.netloc) and bool(parsedURL.scheme)

#The above function works as follows. We check if the scheme is present and there is a value in the network location part
# url = "https://umd.edu/virusinfo"
# urlparse(url)
# ParseResult(scheme='https', netloc='umd.edu', path='/virusinfo', 
# params='', query='', fragment='')

#The function gives all urls
def get_all_urls(url):
    urls = set()
    # domain name of the URL without the protocol (umd.edu in this case)
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    for a_tag in soup.findAll("a"):
        href = a_tag.attrs.get("href")
        if href == "" or href is None:
            #href is empty and we don't need that a element
            continue
        #if the link is not absolute, make it by joining relative to the base
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        #constructing an absolute URL from parsed data
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_url_valid(href):
            #in valid url
            continue
        if href in internal_urls:
            #it is already in the set, so we don't need to add
            continue
        if domain_name not in href:
            #it is an external link. i.e
            # Check if it is already there 
            if href not in external_urls:
#                 print(f"[EXT] External link: {href}")
                external_urls.add(href)
            continue
#         print(".",end=" ")
        print(f"[INT] Internal link: {href}")

        urls.add(href)
        internal_urls.add(href)
    return urls

total_urls_visited = 0
def crawl(url, max_urls=50):
    #Max URL is just to decrease the time if there are a lot of pages.
    #The following code was openly available of github and I found this
    #idea useful to inhibit crawling time
    global total_urls_visited
    total_urls_visited += 1
    try:
        links = get_all_urls(url)
        for link in links:
            if total_urls_visited > max_urls:
                break
            crawl(link, max_urls=max_urls)
    except:
        print("caught")


In [3]:
base_urls = ["https://www.sierranevada.edu/about/covid-19-information/",
"https://www.sru.edu/covid19",
"https://covid19.sonoma.edu/",
"http://www.southeastern.edu/about/safe_campus/covid-19/",
"https://www.se.edu/student-wellness/coronavirus-update/",
"https://www.seu.edu/coronavirus-covid-19-updates/",
"https://www.suu.edu/coronavirus/",
"https://www.suffolk.edu/student-life/health-wellness/coronavirus-health-advisory",
"https://www.stthomas.edu/covid19/",
"https://www.strayer.edu/coronavirus",
"https://srinfo.sulross.edu/covid-19/",
"https://ww1.oswego.edu/oswego-forward/",
"https://www.plattsburgh.edu/covid-19/",
"http://www.sunymaritime.edu/maritime-college-coronavirus-covid-19-update, http://www.sunymaritime.edu/covid-19-update/student-financial-servicescovid-19, http://www.sunymaritime.edu/covid-19-update/student-support-services",
"https://emergency.tcnj.edu/covid-19/",
"https://www.evergreen.edu/covid19",
"https://sites.google.com/trinity.edu/trinityuniversitycovid-19",
"https://www.uagm.edu/covid19",
"http://www.uamont.edu/pages/coronavirus/",
"https://udallas.edu/coronavirus/",
"https://udallas.edu/coronavirus/",
"https://www.evansville.edu/coronavirus/",
"https://www.uog.edu/covid-19/",
"https://www.uhcl.edu/health-alert/",
"https://www.uis.edu/covid-19/",
"https://umdearborn.edu/offices/external-relations/key-issues/um-dearborn-covid-19-response",
"https://www.montevallo.edu/about-um/marcomm/coronavirus-information/",
"https://www.unk.edu/coronavirus/",
"https://ung.edu/covid-19/",
"https://www.untdallas.edu/covid-19-knowledge-center",
"https://unwsp.edu/news/coronavirus",
"https://www.stmary.edu/coronavirus-information",
"https://www.scranton.edu/covid-19/",
"https://www.usf.edu/coronavirus/",
"https://www.uwec.edu/coronavirus-updates/",
"https://www.uww.edu/uhcs/covid19",
"https://www.waldorf.edu/faculty-staff/departments/health-services/covid-19-guidance",
"https://www.walsh.edu/coronavirus.html",
"https://www.weber.edu/coronavirus",
"http://www.wiu.edu/vpas/risk_management_and_emergency_preparedness/coronavirus/",
"https://www.wwu.edu/coronavirus",
"https://www.winthrop.edu/emergency",
"https://www.ycp.edu/about-us/offices-and-departments/communications/coronavirus/"]

In [4]:
for base_url in base_urls:
    internal_urls.add(base_url)
    parsedurl = urlparse(base_url)
    print(parsedurl)
    base_url_main = parsedurl.scheme+"://"+parsedurl.netloc
    print(base_url_main)
    # base_url_text = base_url.split("//",1)[1]
    base_url_text = parsedurl.netloc+parsedurl.path
    print(base_url_text)
    # base_url_text_domain = base_url_text.split("/",1)[0]
    base_url_text_domain = parsedurl.netloc
    print(base_url_text_domain)
    crawl(base_url)
    print("[+] Total External links:", len(external_urls))
    print("[+] Total Internal links:", len(internal_urls))
    print("[+] Total:", len(external_urls) + len(internal_urls))
    count = 0
    last_count = 989874
    immediate_urls = set() #Linked Associated with the current page, https://umd.edu/virusinfo/ in this case

    for i in range(0,4):
        for url in internal_urls:
            if base_url_text in url:
                immediate_urls.add(url)
        count = len(immediate_urls)
        print(count,i)
        if(count == last_count):
            break
        last_count = count
        for immediate_url in immediate_urls:
            try:
                crawl(immediate_url)
                print("[+] NEW Total External links:", len(external_urls))
                print("[+] NEW Total Internal links:", len(internal_urls))
                print("[+] NEW Total:", len(external_urls) + len(internal_urls))
                if len(internal_urls)>1000:
                    break
            except requests.exceptions.ConnectionError:
                continue

    def removeAfterN(yourStr, nth, occurenceOf):
        return occurenceOf.join(yourStr.split(occurenceOf)[:nth])
    
    
    semifinal_immediate_urls = set()

    for immediate_url in immediate_urls:
        stripped_url = removeAfterN(immediate_url,2,"http")


        semifinal_immediate_urls.add(stripped_url)
    #     final_immediate_urls.add(rest)
    #     print(rest)



    for i in semifinal_immediate_urls:
        print(i)
    total_count = len(semifinal_immediate_urls)
    print(total_count)
    
    
    final_immediate_urls = set()
    for semifinal_immediate_url in semifinal_immediate_urls:
        if "tel:" not in semifinal_immediate_url:
            final_immediate_urls.add(semifinal_immediate_url)


    for i in final_immediate_urls:
        print(i)
    total_count = len(final_immediate_urls)
    print(total_count)       
    
    
    
    parent_folder = "Masters" #Change based on folder
    name_of_folder = f'{base_url_text_domain}_Screenshots'
    options = webdriver.ChromeOptions()
    options.headless = True
    driver = webdriver.Chrome(options=options,executable_path='chromedriver.exe') # #Local Path of chrome driver
    if not os.path.exists(f'{parent_folder}\\{name_of_folder}'):
        os.makedirs(f'{parent_folder}\\{name_of_folder}')
    
    
    
    webpage_save = set()
    webpage_save_count = 0
    webpage_cannot_save = 0
    for i in final_immediate_urls:
        if ' ' in i:
            continue
        try:
            code=urlopen(i).getcode()
            webpage_save.add(i)
            webpage_save_count = webpage_save_count+1
            print(f"{code} can_save - {i}")
        except (urllib.error.HTTPError,urllib.error.URLError) as e:
            print("discarded")
            webpage_cannot_save = webpage_cannot_save+1
            continue

    print(f'Webpages saved as HTML :{webpage_save_count}')
    print(f'Webpages unable to save as HTML :{webpage_cannot_save}')

    
    pdf_count = 0
    pdf_urls = set()
    for webpage in webpage_save:
        try:
            if webpage[-4:] == ".pdf":
                url = webpage
                file_name = url.replace(base_url,'')
                file_name = file_name.replace('/','')
                if file_name == '':
                    file_name = "Home"
                pdf_urls.add(webpage)
                print(webpage)
                r = requests.get(url, stream=True)
                with open(f'{parent_folder}\\{name_of_folder}\\{file_name}.pdf', 'wb') as fd:
                    for chunk in r.iter_content(2000):
                        fd.write(chunk)
                pdf_count = pdf_count + 1
        except:
            print("caught")

    print(f"Pdfs saved: {pdf_count}")
    
    
    
    final_immediate_urls = final_immediate_urls - pdf_urls
    webpage_save = webpage_save - pdf_urls
    web_save_count = len(webpage_save)
    total_count = len(final_immediate_urls)
    print(total_count) 
    
    
    
    final_webpage_save_count = webpage_save_count - pdf_count
    print(final_webpage_save_count)
    
    
    webpage_save_counter_loop = 0
    for webpage in webpage_save:
        try:
            url = webpage
            file_name = url.replace(base_url,'')
            file_name = file_name.replace('/','')
            if file_name == '':
                    file_name = "Home"
            urllib.request.urlretrieve (webpage, f"{parent_folder}\\{name_of_folder}\\{file_name}.html")
            webpage_save_counter_loop = webpage_save_counter_loop+1
            print(f'{webpage_save_counter_loop} of {final_webpage_save_count} - {webpage}')
        except:
            print("caught")

    print("Saving Completed")
    
    
    count = 0
    for i in final_immediate_urls:
        try:
            url = i
            file_name = url.replace(base_url,'')
            file_name = file_name.replace('/','')
            if file_name == '':
                    file_name = "Home"
            count = count + 1
            print(f"{count} of {total_count}")
            print(f'Visiting {base_url_text}/{file_name}')
            print(f'...Taking a screenshot')
            driver.get(url)
            S = lambda X: driver.execute_script('return document.body.parentNode.scroll'+X)
            driver.set_window_size(S('Width'),S('Height')) # May need manual adjustment                                                                                                                
            driver.find_element_by_tag_name('body').screenshot(f'{parent_folder}\\{name_of_folder}\\{file_name}.png')
            print(f'Screenshot of {file_name} page taken! \n')
        except:
            print("caught")
    #     print("Saving HTML")
    #     pyautogui.hotkey('ctrl', 's')
    #     time.sleep(1)
    #     pyautogui.typewrite(f'{file_name}.html')
    #     pyautogui.hotkey('enter')
    driver.quit()

    print(f'Task Completed! Files stored in the {parent_folder}\\{name_of_folder} Folder')

ParseResult(scheme='https', netloc='www.sierranevada.edu', path='/about/covid-19-information/', params='', query='', fragment='')
https://www.sierranevada.edu
www.sierranevada.edu/about/covid-19-information/
www.sierranevada.edu
[+] Total External links: 0
[+] Total Internal links: 1
[+] Total: 1
1 0
[+] NEW Total External links: 0
[+] NEW Total Internal links: 1
[+] NEW Total: 1
1 1
https://www.sierranevada.edu/about/covid-19-information/
1
https://www.sierranevada.edu/about/covid-19-information/
1
discarded
Webpages save as HTML :0
Webpages unable to save as HTML :1
Pdfs saved: 0
1
0
Saving Completed
1 of 1
Visiting www.sierranevada.edu/about/covid-19-information//Home
...Taking a screenshot
Screenshot of Home page taken! 

Task Completed! Files stored in the Masters\www.sierranevada.edu_Screenshots Folder
ParseResult(scheme='https', netloc='www.sru.edu', path='/covid19', params='', query='', fragment='')
https://www.sru.edu
www.sru.edu/covid19
www.sru.edu
[INT] Internal link: https:

[INT] Internal link: http://www.sru.edu/academics/majors-and-minors/information-systems/information-systems-major-requirements
[INT] Internal link: http://www.sru.edu/academics/majors-and-minors/information-systems/information-systems-minor-requirements
[INT] Internal link: http://www.sru.edu/academics/colleges-and-departments/cob/school-of-business/healthcare-administration-and-information-systems
[INT] Internal link: http://www.sru.edu/academics/majors-and-minors/information-systems/documents/programs/curriculumguides/infosystemsminor.pdf
[INT] Internal link: http://www.sru.edu/academics/majors-and-minors/information-systems/documents/programs/factsheets/undergraduate/FS-InfoSystems.pdf
[INT] Internal link: http://www.sru.edu/404
[INT] Internal link: http://www.sru.edu/academics/majors-and-minors/information-systems/documents/programs/curriculumguides/infosystems.pdf
[INT] Internal link: http://www.sru.edu/academics/majors-and-minors/information-systems/documents/programs/curriculumg

[INT] Internal link: http://www.sru.edu/admissions/financial-aid/types-of-aid/scholarships
[INT] Internal link: http://www.sru.edu/admissions/financial-aid/types-of-aid/scholarships/first-year-scholarships
[INT] Internal link: http://www.sru.edu/admissions/financial-aid/types-of-aid/scholarships/distinguished-scholarships-and-fellowships-program
[INT] Internal link: http://www.sru.edu/admissions/financial-aid/types-of-aid/scholarships/returning-undergraduate-scholarships
[INT] Internal link: http://www.sru.edu/admissions/financial-aid/types-of-aid/scholarships/graduate-scholarships
[INT] Internal link: http://www.sru.edu/admissions/financial-aid/types-of-aid/scholarships/scholarships-for-students-with-disabilities
[INT] Internal link: http://www.sru.edu/admissions/financial-aid/types-of-aid/scholarships/external-scholarships
[INT] Internal link: http://www.sru.edu/admissions/financial-aid/types-of-aid
[INT] Internal link: http://www.sru.edu/transfer
[INT] Internal link: http://www.sru.

[INT] Internal link: https://www.sru.edu/covid19/docs/heerf-students-certification-agreement-form.pdf
[INT] Internal link: https://www.sru.edu/covid19/docs/cares-faq-archive.pdf
[INT] Internal link: https://www.sru.edu/covid19/docs/letter-to-students-cares-disclosures.pdf
[INT] Internal link: https://www.sru.edu/covid19/docs/letter-to-students-cares-disclosures-2.pdf
[+] NEW Total External links: 139
[+] NEW Total Internal links: 460
[+] NEW Total: 599
[+] NEW Total External links: 139
[+] NEW Total Internal links: 460
[+] NEW Total: 599
[INT] Internal link: https://www.sru.edu/offices/iats/support-services
[INT] Internal link: https://www.sru.edu/documents/offices/IATS/ZOOM.pdf
[INT] Internal link: https://www.sru.edu/technology-business-continuity
[INT] Internal link: https://www.sru.edu/covid19/docs/remote-work-tips.pdf
[INT] Internal link: https://www.sru.edu/documents/life-at-sru/health-and-wellness/COVID-19/travel-faq.pdf
[+] NEW Total External links: 147
[+] NEW Total Internal l

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


...

Screenshot of letter-june22-espanol.html page taken! 

35 of 36
Visiting www.weber.edu/coronavirus/letter-july1.html
...Taking a screenshot
Screenshot of letter-july1.html page taken! 

36 of 36
Visiting www.weber.edu/coronavirus/letter-june22.html
...Taking a screenshot
Screenshot of letter-june22.html page taken! 

Task Completed! Files stored in the Masters\www.weber.edu_Screenshots Folder
ParseResult(scheme='http', netloc='www.wiu.edu', path='/vpas/risk_management_and_emergency_preparedness/coronavirus/', params='', query='', fragment='')
http://www.wiu.edu
www.wiu.edu/vpas/risk_management_and_emergency_preparedness/coronavirus/
www.wiu.edu
[INT] Internal link: http://www.wiu.edu/parents_and_family/
[INT] Internal link: http://www.wiu.edu/visit/
[INT] Internal link: http://www.wiu.edu/global_education/global.php
[INT] Internal link: http://www.wiu.edu/libraries/
[INT] Internal link: http://www.wiu.edu/registrar/courses.php
[INT] Internal link: http://www.wiu.edu/table_of_contents/


[+] NEW Total External links: 2018
[+] NEW Total Internal links: 3371
[+] NEW Total: 5389
5 1
https://www.wwu.edu/coronavirus
https://www.wwu.edu/coronavirus/campus-life
https://www.wwu.edu/coronavirus/workplace
https://www.wwu.edu/coronavirus/families-visitors
https://www.wwu.edu/coronavirus/students
5
https://www.wwu.edu/coronavirus
https://www.wwu.edu/coronavirus/campus-life
https://www.wwu.edu/coronavirus/workplace
https://www.wwu.edu/coronavirus/families-visitors
https://www.wwu.edu/coronavirus/students
5
200 can_save - https://www.wwu.edu/coronavirus
200 can_save - https://www.wwu.edu/coronavirus/campus-life
200 can_save - https://www.wwu.edu/coronavirus/workplace
200 can_save - https://www.wwu.edu/coronavirus/families-visitors
200 can_save - https://www.wwu.edu/coronavirus/students
Webpages save as HTML :5
Webpages unable to save as HTML :0
Pdfs saved: 0
5
5
1 of 5 - https://www.wwu.edu/coronavirus/students
2 of 5 - https://www.wwu.edu/coronavirus/workplace
3 of 5 - https://www.

[INT] Internal link: https://www.winthrop.edu/emergency/ /activitiesandorganizations
[INT] Internal link: https://www.winthrop.edu/emergency/
[INT] Internal link: https://www.winthrop.edu/academic-affairs/index.aspx
[INT] Internal link: https://www.winthrop.edu/facilitymgmt/
[+] NEW Total External links: 2042
[+] NEW Total Internal links: 3474
[+] NEW Total: 5516
37 1
[+] NEW Total External links: 2042
[+] NEW Total Internal links: 3474
[+] NEW Total: 5516
37 2
https://www.winthrop.edu/emergency/classroom-emergency-response.aspx
https://www.winthrop.edu/emergency/civil-disturbance.aspx
https://www.winthrop.edu/emergency/inclement.aspx
https://www.winthrop.edu/emergency/macfeat-early-childhood-school.aspx
https://www.winthrop.edu/emergency/utilities-system-malfunction.aspx
https://www.winthrop.edu/emergency/tornado.aspx
https://www.winthrop.edu/emergency/shelter-in-place.aspx
https://www.winthrop.edu/emergency/residence-hall-plan-earthquake.aspx
https://www.winthrop.edu/emergency/classr

12 of 36 - https://www.winthrop.edu/emergency/crisis-communication.aspx
13 of 36 - https://www.winthrop.edu/emergency/fire-response.aspx
14 of 36 - https://www.winthrop.edu/emergency/fire.aspx
15 of 36 - https://www.winthrop.edu/emergency/shelter-locations.aspx
16 of 36 - https://www.winthrop.edu/emergency/major-emergency.aspx
17 of 36 - https://www.winthrop.edu/emergency/active-shooter.aspx
18 of 36 - https://www.winthrop.edu/emergency/coronavirus.aspx
19 of 36 - https://www.winthrop.edu/emergency/training-resources.aspx
20 of 36 - https://www.winthrop.edu/emergency/earthquake.aspx
21 of 36 - https://www.winthrop.edu/emergency/critical-incident-safety-tips.aspx
22 of 36 - https://www.winthrop.edu/emergency/residence-hall-plan-nuclear-accident.aspx
23 of 36 - https://www.winthrop.edu/emergency/terrorist.aspx
24 of 36 - https://www.winthrop.edu/emergency/residence-hall-plan-tornado.aspx
25 of 36 - https://www.winthrop.edu/emergency/building-evacuation.aspx
26 of 36 - https://www.winthro

[INT] Internal link: https://www.ycp.edu/about-us/offices-and-departments/communications/coronavirus/employees-faq/tel:7178467788
[+] NEW Total External links: 2081
[+] NEW Total Internal links: 4054
[+] NEW Total: 6135
8 1
[+] NEW Total External links: 2081
[+] NEW Total Internal links: 4054
[+] NEW Total: 6135
8 2
https://www.ycp.edu/about-us/offices-and-departments/communications/coronavirus/employees-faq/
https://www.ycp.edu/about-us/offices-and-departments/communications/coronavirus/
https://www.ycp.edu/about-us/offices-and-departments/communications/coronavirus/tel:7178156642
https://www.ycp.edu/about-us/offices-and-departments/communications/coronavirus/tel:7178151282
https://www.ycp.edu/about-us/offices-and-departments/communications/coronavirus/tel:7178467788
https://www.ycp.edu/about-us/offices-and-departments/communications/coronavirus/employees-faq/tel:7178467788
https://www.ycp.edu/about-us/offices-and-departments/communications/coronavirus/contacts/
https://www.ycp.edu/ab

In [5]:
print("DONE")

DONE
